In [22]:
import pandas as pd

#import library Scikit-learn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

#import pickle
import pickle

#import library untuk masing-masing algoritma
from sklearn.naive_bayes import GaussianNB

from sklearn import tree

from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb

#import metrik yang dibutuhkan untuk cross validation
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

Read dataset train

In [2]:
df = pd.read_csv("G:\\Datasets\\klasifikasi\\train.csv", delimiter = "|")

# Tujuan

Memprediksi ada tidak nya transaksi fraud

# Eksplorasi Data

Memeriksa info dari dataset untuk mengidentifikasi adanya data NULL

In [3]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1879 entries, 0 to 1878
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   trustLevel                 1879 non-null   int64  
 1   totalScanTimeInSeconds     1879 non-null   int64  
 2   grandTotal                 1879 non-null   float64
 3   lineItemVoids              1879 non-null   int64  
 4   scansWithoutRegistration   1879 non-null   int64  
 5   quantityModifications      1879 non-null   int64  
 6   scannedLineItemsPerSecond  1879 non-null   float64
 7   valuePerSecond             1879 non-null   float64
 8   lineItemVoidsPerPosition   1879 non-null   float64
 9   fraud                      1879 non-null   int64  
dtypes: float64(4), int64(6)
memory usage: 146.9 KB


,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0


Setelah diperiksa ternyata tidak ada data null di dataset

Selain itu, kami menyimpulkan ahwa semua atribut yang ada relevan dengan tujuan prediksi. Oleh karena itu tidak ada atribut yang di drop.

In [23]:
df.trustLevel.value_counts()

2    347
1    332
3    318
5    302
6    291
4    289
Name: trustLevel, dtype: int64

In [5]:
dfFraud = df[df.fraud==1]

In [6]:
dfFraud.info()
dfFraud.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104 entries, 5 to 1875
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   trustLevel                 104 non-null    int64  
 1   totalScanTimeInSeconds     104 non-null    int64  
 2   grandTotal                 104 non-null    float64
 3   lineItemVoids              104 non-null    int64  
 4   scansWithoutRegistration   104 non-null    int64  
 5   quantityModifications      104 non-null    int64  
 6   scannedLineItemsPerSecond  104 non-null    float64
 7   valuePerSecond             104 non-null    float64
 8   lineItemVoidsPerPosition   104 non-null    float64
 9   fraud                      104 non-null    int64  
dtypes: float64(4), int64(6)
memory usage: 8.9 KB


,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
5,1,770,11.09,11,5,2,0.033766,0.014403,0.423077,1
37,2,1397,62.59,7,9,4,0.017895,0.044803,0.280000,1
53,1,1385,34.68,5,8,4,0.019495,0.025040,0.185185,1
91,1,996,71.94,7,0,1,0.027108,0.072229,0.259259,1
97,2,1305,87.65,7,8,4,0.021456,0.067165,0.250000,1


In [7]:
dfFraud.trustLevel.value_counts()

1    89
2    15
Name: trustLevel, dtype: int64

# Kelas Target

In [8]:
le = preprocessing.LabelEncoder()
le.fit(df.fraud)
Y = df.fraud 
X = df.drop("fraud",axis=1)

In [9]:
list(le.classes_)

[0, 1]

# Split Dataset

In [10]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=123)


with open('G:\\Datasets\\klasifikasi\\fraud_train_columns.pickle', 'wb') as fp:
    pickle.dump(X_train.columns, fp)

# Gaussian NB

In [11]:
clf = GaussianNB()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))

Akurasi 0.8670212765957447
              precision    recall  f1-score   support

           0       1.00      0.86      0.93       361
           1       0.23      1.00      0.38        15

    accuracy                           0.87       376
   macro avg       0.62      0.93      0.65       376
weighted avg       0.97      0.87      0.90       376

[[311  50]
 [  0  15]]


In [12]:
def classification_report_with_accuracy_score(y_true, y_pred):    
    print(classification_report(y_true, y_pred)) 
    print(confusion_matrix(y_true, y_pred))
    return accuracy_score(y_true, y_pred) 
clf = GaussianNB()
scores = cross_val_score(clf, X=X, y=Y, cv=3, scoring = make_scorer(classification_report_with_accuracy_score))
print(scores)

              precision    recall  f1-score   support

           0       1.00      0.82      0.90       592
           1       0.23      0.94      0.38        35

    accuracy                           0.82       627
   macro avg       0.61      0.88      0.64       627
weighted avg       0.95      0.82      0.87       627

[[484 108]
 [  2  33]]
              precision    recall  f1-score   support

           0       1.00      0.83      0.91       592
           1       0.25      0.94      0.39        34

    accuracy                           0.84       626
   macro avg       0.62      0.89      0.65       626
weighted avg       0.96      0.84      0.88       626

[[494  98]
 [  2  32]]
              precision    recall  f1-score   support

           0       1.00      0.84      0.91       591
           1       0.27      1.00      0.42        35

    accuracy                           0.85       626
   macro avg       0.63      0.92      0.67       626
weighted avg       0.96     

# Decision tree

In [13]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))

Akurasi 0.9787234042553191
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       361
           1       0.77      0.67      0.71        15

    accuracy                           0.98       376
   macro avg       0.88      0.83      0.85       376
weighted avg       0.98      0.98      0.98       376

[[358   3]
 [  5  10]]


In [14]:
feature_importances = pd.DataFrame(clf.feature_importances_,index = X_train.columns, columns=['importance']).sort_values('importance',ascending=False)

print(feature_importances)

                           importance
scannedLineItemsPerSecond    0.251054
totalScanTimeInSeconds       0.215100
trustLevel                   0.196980
lineItemVoids                0.088886
lineItemVoidsPerPosition     0.082961
grandTotal                   0.049077
valuePerSecond               0.046552
scansWithoutRegistration     0.042836
quantityModifications        0.026555


In [15]:
def classification_report_with_accuracy_score(y_true, y_pred):    
    print(classification_report(y_true, y_pred)) 
    print(confusion_matrix(y_true, y_pred))
    return accuracy_score(y_true, y_pred) 
clf = tree.DecisionTreeClassifier()
scores = cross_val_score(clf, X=X, y=Y, cv=3, scoring = make_scorer(classification_report_with_accuracy_score))
print(scores)


              precision    recall  f1-score   support

           0       0.98      0.98      0.98       592
           1       0.71      0.63      0.67        35

    accuracy                           0.96       627
   macro avg       0.84      0.81      0.82       627
weighted avg       0.96      0.96      0.96       627

[[583   9]
 [ 13  22]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       592
           1       0.63      0.71      0.67        34

    accuracy                           0.96       626
   macro avg       0.81      0.84      0.82       626
weighted avg       0.96      0.96      0.96       626

[[578  14]
 [ 10  24]]
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       591
           1       0.65      0.49      0.56        35

    accuracy                           0.96       626
   macro avg       0.81      0.74      0.77       626
weighted avg       0.95     

# Random forest

In [16]:
clf = RandomForestClassifier(n_estimators=50, random_state=123)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))


Akurasi 0.9867021276595744
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       361
           1       1.00      0.67      0.80        15

    accuracy                           0.99       376
   macro avg       0.99      0.83      0.90       376
weighted avg       0.99      0.99      0.99       376

[[361   0]
 [  5  10]]


In [17]:
feature_importances = pd.DataFrame(clf.feature_importances_,index = X_train.columns, columns=['importance']).sort_values('importance',ascending=False)

print(feature_importances)


                           importance
trustLevel                   0.275139
totalScanTimeInSeconds       0.153191
scannedLineItemsPerSecond    0.149126
lineItemVoidsPerPosition     0.099001
valuePerSecond               0.087869
grandTotal                   0.081010
lineItemVoids                0.072523
scansWithoutRegistration     0.056474
quantityModifications        0.025666


In [18]:
def classification_report_with_accuracy_score(y_true, y_pred):    
    print(classification_report(y_true, y_pred)) 
    print(confusion_matrix(y_true, y_pred))
    return accuracy_score(y_true, y_pred) 
clf = RandomForestClassifier(n_estimators=50, random_state=123)
scores = cross_val_score(clf, X=X, y=Y, cv=3, scoring = make_scorer(classification_report_with_accuracy_score))
print(scores)


              precision    recall  f1-score   support

           0       0.98      0.99      0.99       592
           1       0.83      0.69      0.75        35

    accuracy                           0.97       627
   macro avg       0.90      0.84      0.87       627
weighted avg       0.97      0.97      0.97       627

[[587   5]
 [ 11  24]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       592
           1       0.96      0.76      0.85        34

    accuracy                           0.99       626
   macro avg       0.97      0.88      0.92       626
weighted avg       0.99      0.99      0.98       626

[[591   1]
 [  8  26]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       591
           1       0.92      0.34      0.50        35

    accuracy                           0.96       626
   macro avg       0.94      0.67      0.74       626
weighted avg       0.96     

# XGBoost

In [19]:
clf = xgb.XGBClassifier(objective = "multi:softprob", num_class = 4, eval_metric = "mlogloss", max_depth = 24, gamma=0.1, subsample = 0.90,learning_rate=0.01, n_estimators = 10, nthread=-1)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))

Akurasi 0.9920212765957447
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       361
           1       1.00      0.80      0.89        15

    accuracy                           0.99       376
   macro avg       1.00      0.90      0.94       376
weighted avg       0.99      0.99      0.99       376



G:\Anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [20]:
feature_importances = pd.DataFrame(clf.feature_importances_,index = X_train.columns, columns=['importance']).sort_values('importance',ascending=False)

print(feature_importances)

                           importance
trustLevel                   0.414826
scannedLineItemsPerSecond    0.132589
totalScanTimeInSeconds       0.111765
lineItemVoidsPerPosition     0.103356
lineItemVoids                0.072808
valuePerSecond               0.067117
scansWithoutRegistration     0.045965
grandTotal                   0.026483
quantityModifications        0.025090


In [21]:
def classification_report_with_accuracy_score(y_true, y_pred):    
    print(classification_report(y_true, y_pred)) 
    print(confusion_matrix(y_true, y_pred))
    return accuracy_score(y_true, y_pred) 
clf = xgb.XGBClassifier(objective = "multi:softprob", num_class = 3, eval_metric = "mlogloss", max_depth = 24, gamma=0.1, subsample = 0.90,learning_rate=0.01, n_estimators = 10, nthread=-1)
scores = cross_val_score(clf, X=X, y=Y, cv=5, scoring = make_scorer(classification_report_with_accuracy_score))
print(scores)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       355
           1       0.83      0.71      0.77        21

    accuracy                           0.98       376
   macro avg       0.91      0.85      0.88       376
weighted avg       0.97      0.98      0.98       376

[[352   3]
 [  6  15]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       355
           1       0.88      0.67      0.76        21

    accuracy                           0.98       376
   macro avg       0.93      0.83      0.87       376
weighted avg       0.97      0.98      0.97       376

[[353   2]
 [  7  14]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       355
           1       0.88      0.67      0.76        21

    accuracy                           0.98       376
   macro avg       0.93      0.83      0.87       376
weighted avg       0.97     

G:\Anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
G:\Anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
G:\Anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is depre

[[353   2]
 [  7  14]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       355
           1       0.80      0.57      0.67        21

    accuracy                           0.97       376
   macro avg       0.89      0.78      0.82       376
weighted avg       0.97      0.97      0.97       376

[[352   3]
 [  9  12]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       355
           1       0.77      0.50      0.61        20

    accuracy                           0.97       375
   macro avg       0.87      0.75      0.79       375
weighted avg       0.96      0.97      0.96       375



G:\Anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
G:\Anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[[352   3]
 [ 10  10]]
[0.97606383 0.97606383 0.97606383 0.96808511 0.96533333]
